# Développez un chatbot pour réserver des vacances

## Sommaire

* [I. Préambule](#I)
* [II. Chargement des données](#II)
* [III. Mise en place de LUIS](#III)
    * [1. Création de l'application](#III1)
    * [2. Entraînement de l'application](#III2)
        
## I. Préambule<a class="anchor" id="I"></a>

Fly Me est une agence qui propose des voyages clé en main pour les particuliers ou les professionnels. 

Ils ont lancé un projet ambitieux de développement d’un chatbot pour aider les utilisateurs à choisir une offre de voyage.

La première étape de ce projet est de construire un MVP qui aidera les employés de Fly Me à réserver facilement un billet d’avion pour leurs vacances.

Ce premier MVP nous permettra de pouvoir tester rapidement et à grande échelle le concept et les performances du chatbot.

Comme ce projet est itératif, nous avons limité les fonctionnalités de la V1 du chatbot. La V1 devra pouvoir identifier dans la demande de l’utilisateur les cinq éléments suivants :
* Ville de départ
* Ville de destination
* Date aller souhaitée du vol
* Date retour souhaitée du vol
* Budget maximum pour le prix total des billets.

Si un des éléments est manquant, le chatbot devra pouvoir poser les questions pertinentes, en anglais, à l’utilisateur pour comprendre complètement sa demande. Lorsque le chatbot pense avoir compris tous les éléments de la demande de l’utilisateur, il doit pouvoir reformuler la demande de l’utilisateur et lui demander de valider sa compréhension.

In [1]:
# Import libraries
import os
import uuid
import time
import json

# Math libraries to process the data 
import numpy as np 
import pandas as pd

# Libraries
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject

# Graph libraries to produce graphs 
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly.express as px

pd.options.mode.chained_assignment = None

## II. Chargement des données<a class="anchor" id="II"></a>

Nous allons charger les données en mémoire.

In [2]:
def entity_char(entity_name, frame):
    # Load entity part of the frame
    entities = frame['labels']['frames'][0]['info']
    str_entire = frame['labels']['frames'][0]['info'][entity_name][0]['val'].lower()
    return str_entire.split('.')[0]

def entity_data(entity_char, text, entity_name):
    start_index = max(text.find(entity_char), 0)
    end_char_index = max(text.find(entity_char) + len(entity_char) - 1, 0)
    return {'startCharIndex': start_index, 'endCharIndex': end_char_index, 'entityName': entity_name}

def entity_data_dict(frame, pf_entities, intent_name):
    text = frame['text'].lower()
    entities = frame['labels']['frames'][0]['info']    
    entity_list = []
    for entity_name in pf_entities:
        # Check if entities exist
        if entity_name in entities.keys():
            # Check if value is not -1
            if entity_char(entity_name, frame)!='-1':
                entity_list.append(entity_data(entity_char(entity_name, frame), text, entity_name))
    dict_output = {
        "text": text,
        "intentName": intent_name,
        "entityLabels": entity_list
    }
    return dict_output

dataframe = pd.read_json("data/frames.json")
pf_entities = ['or_city','dst_city','str_date','end_date','budget']
utterances = [
    entity_data_dict(
        dataframe['turns'][x][0], pf_entities, "BookFlightIntent"
    ) for x in range(0, len(dataframe['turns']))
]

if not os.path.exists("data/utterances_train.json"):
    utterances_train = utterances[:-107]

    with open('data/utterances_train.json', 'w') as f:
        f.write(json.dumps(utterances_train))
else:
    with open("data/utterances_train.json") as f:
        utterances_train = json.load(f)

if not os.path.exists("data/utterances_test.json"):
    utterances_test = utterances[-107:]

    with open('data/utterances_test.json', 'w') as f:
        f.write(json.dumps(utterances_test))
else:
    with open("data/utterances_test.json") as f:
        utterances_test = json.load(f)

## III. Mise en place de LUIS<a class="anchor" id="III"></a>

### 1. Création de l'application<a class="anchor" id="III1"></a>

Language Understanding (LUIS) nous permet d’appliquer un traitement en langage naturel au texte en langage naturel des conversations d’un utilisateur afin d’en prédire le sens général, et d’en extraire des informations détaillées et pertinentes.

Une application LUIS stocke le modèle de traitement en langage naturel contenant les intentions, les entités et les exemples d'énoncés.

In [6]:
# Create variables
authoring_key = "79b60d071a594d15b2a83e30ac9dc8c4"
authoring_endpoint = "https://chatbotluisoc-authoring.cognitiveservices.azure.com/"
prediction_key = "c4d06d80ea7b4e608eadfc44b51ec3bb"
prediction_endpoint = "https://chatbot-luis-oc.cognitiveservices.azure.com/"
app_name = "P10 chatbot " + str(uuid.uuid4())
version_id = "0.1"
intent_name = "BookFlightIntent"

# Create client
client = LUISAuthoringClient(authoring_endpoint, CognitiveServicesCredentials(authoring_key))

# Define app basics
app_definition = ApplicationCreateObject(name=app_name, initial_version_id=version_id, culture='en-us')

# Create app
app_id = client.apps.add(app_definition)

# Get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID 476ae015-dac4-44ce-9dcc-ca1da0154e3b


L’objet principal dans un modèle d’application LUIS est l’intention. L’intention s’aligne sur un regroupement d’intentions d’énoncés utilisateur. Un utilisateur peut poser une question ou émettre un énoncé en souhaitant obtenir une réponse prévue particulière d’un bot (ou d’une autre application cliente). Réserver un billet d’avion, demander quelle est la météo dans une ville de destination et demander des informations de contact pour un service client sont des exemples d’intentions.

In [7]:
# Add new intention
client.model.add_intent(app_id, version_id, intent_name)

'03af29bd-6563-41b2-bee4-498c9d08374f'

Les entités, elles, ne soient pas obligatoires, elles sont présentes dans la plupart des applications. L’entité extrait des informations à partir de l’énoncé utilisateur, qui sont nécessaires pour répondre à l’intention de l’utilisateur. Il existe plusieurs types d’entités prédéfinies et personnalisées, chacune avec leurs propres modèles DTO (Data Transformation Object). Les entités prédéfinies courantes à ajouter à notre application incluent number, datetimeV2, geographyV2 et ordinal.

Il est important de savoir que les entités ne sont pas marquées avec une intention. Elles peuvent s’appliquer à de nombreuses intentions. Seuls les exemples d’énoncés utilisateur sont marqués pour une intention unique spécifique.

In [8]:
# Add Entities
model_id = client.model.add_entity(app_id, version_id, name="or_city")
model_id = client.model.add_entity(app_id, version_id, name="dst_city")
model_id = client.model.add_entity(app_id, version_id, name="end_date")
model_id = client.model.add_entity(app_id, version_id, name="str_date")
model_id = client.model.add_entity(app_id, version_id, name="budget")

Pour déterminer l’intention d’un énoncé et extraire des entités, l’application a besoin d’exemples d’énoncés. Les exemples doivent cibler une intention spécifique et unique, et doivent marquer toutes les entités personnalisées. Les entités prédéfinies n’ont pas besoin d’être marquées.

In [10]:
# Add examples
for x in range(0, len(utterances_train)):
    try:
        client.examples.add(app_id, version_id, utterances_train[x])
    except Exception as e:
        pass

### 2. Entraînement de l'application<a class="anchor" id="III2"></a>

Maintenant que le modèle est créé, l’application LUIS va être entraînée pour cette version du modèle. Un modèle entraîné peut être utilisé dans un conteneur ou publié dans les emplacements intermédiaires ou produits.

Un petit modèle sera entraîné très rapidement. Pour les applications de niveau production, l’entraînement de l’application va devoir inclure un appel d’interrogation à la méthode get_status pour déterminer si l’entraînement a réussi. Tous les objets doivent réussir pour que l’entraînement soit considéré comme terminé.

In [11]:
print("We'll start training your app...")

async_training = client.train.train_version(app_id, version_id)
is_trained = async_training.status == "UpToDate"

trained_status = ["UpToDate", "Success"]
while not is_trained:
    time.sleep(1)
    status = client.train.get_status(app_id, version_id)
    is_trained = all(m.details.status in trained_status for m in status)
    
print("Training done")

We'll start training your app...
Training done


Nous allons publier l’application LUIS. Cela va permettre de la publier à l’emplacement spécifié au point de terminaison. Notre application cliente utilise ce point de terminaison afin d’envoyer des énoncés utilisateur pour la prédiction de l’intention et l’extraction d’entité.

In [12]:
print("We'll start publishing your app...")
client.apps.update_settings(app_id, is_public=True)
publish_result = client.apps.publish(app_id, version_id, is_staging=False)
publish_result.as_dict()
endpoint = publish_result.endpoint_url + "?subscription-key=" + authoring_key + "&q="
print("Your app is published. You can now go to test it on\n{}".format(endpoint))

We'll start publishing your app...
Your app is published. You can now go to test it on
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/476ae015-dac4-44ce-9dcc-ca1da0154e3b?subscription-key=79b60d071a594d15b2a83e30ac9dc8c4&q=


Nous allons créer une requête à adresser au runtime de prédiction. Cette requête va nous retourner un objet PredictionResponse.

In [23]:
runtime_credentials = CognitiveServicesCredentials(prediction_key)
client_runtime = LUISRuntimeClient(endpoint=prediction_endpoint, credentials=runtime_credentials)

# Test with one utterance
query = "looking to go from san francisco to marseille. book me for september 18 to 22. let me know if its more than 2800 because thats all i can afford"
result = client_runtime.prediction.resolve(app_id, query, verbose=False)

print("Query: {}".format(query))
print("Detected entities:")
for entity in result.entities:
    print(
        "\t-> Entity '{}' (type: {}, score:{:d}%)".format(
            entity.entity,
            entity.type,
            int(entity.additional_properties['score']*100)
        ))

Query: looking to go from san francisco to marseille. book me for september 18 to 22. let me know if its more than 2800 because thats all i can afford
Detected entities:
	-> Entity '2800' (type: budget, score:99%)
	-> Entity 'marseille' (type: dst_city, score:99%)
	-> Entity '22' (type: end_date, score:99%)
	-> Entity 'san francisco' (type: or_city, score:99%)
	-> Entity 'september 18' (type: str_date, score:99%)


Calculons maintenant le score moyen de chaque entité dans notre fichier de test.

In [34]:
cpt_budget = 0
score_budget = 0
cpt_dst_city = 0
score_dst_city = 0
cpt_or_city = 0
score_or_city = 0
cpt_end_date = 0
score_end_date = 0
cpt_str_date = 0
score_str_date = 0

for text in utterances_test:
    query = text['text']
    result = client_runtime.prediction.resolve(app_id, query, verbose=False)
    for entity in result.entities:
        if "budget" == entity.type:
            cpt_budget += 1
            score_budget += entity.additional_properties["score"] * 100
        if "dst_city" == entity.type:
            cpt_dst_city += 1
            score_dst_city += entity.additional_properties["score"] * 100
        if "end_date" == entity.type:
            cpt_end_date += 1
            score_end_date += entity.additional_properties["score"] * 100
        if "or_city" == entity.type:
            cpt_or_city += 1
            score_or_city += entity.additional_properties["score"] * 100
        if "str_date" == entity.type:
            cpt_str_date += 1
            score_str_date += entity.additional_properties["score"] * 100
print("Mean budget score: {}".format(score_budget / cpt_budget))
print("Mean dst_city score: {}".format(score_dst_city / cpt_dst_city))
print("Mean end_date score: {}".format(score_end_date / cpt_end_date))
print("Mean or_city score: {}".format(score_or_city / cpt_or_city))
print("Mean str_date score: {}".format(score_str_date / cpt_str_date))

Mean budget score: 98.06385515384615
Mean dst_city score: 97.92328239726027
Mean end_date score: 99.550915
Mean or_city score: 96.71587676785714
Mean str_date score: 86.69668266666666
